# Setup for Amazon Bedrock using Local VS Code

이 노트북에서는 개인 노트북, 온프레미스 서버등에서 VS Code 로 Amazon Bedrock API 를 사용할 수 있는 가이드를 제공 합니다. 필수적으로 AWS IAM User, Role 을 가지고 있어야 합니다.
### Test Environment: Apple M1 Pro, Ventura 13.5.2 및 윈도우서버 2022
---

#### Reference
- Bedrock user guide
    - https://docs.aws.amazon.com/bedrock/latest/userguide/what-is-service.html



# 1. Pre-requisite (선수 항목)

## 1.1. 로컬 환경 구성
아래와 같이 VS Code 가 있어야 하고, 이미 설치가 되어 있으면 지나 가셔도 됩니다.

- VS Code 설치
    - [Code editing.Redefined](https://code.visualstudio.com/)
- VS Code Extension 설치 (아래 링크 참조 하세요)
    - Install Python extension and Jupyter
        - [Data Science in VS Code tutorial](https://code.visualstudio.com/docs/datascience/data-science-tutorial)
- Anaconda 설치
    - 아래는 Anaconda 의 최소 버전 (Conda, Python 만 포함) 설치
    - [Miniconda](https://docs.conda.io/en/latest/miniconda.html)
- Git 설치
    - [Download for macOS](https://git-scm.com/download/mac)


## 1.2. AWS User, Role 생성 및 권한 추가

연결 설정은 블로그: [Run your TensorFlow job on Amazon SageMaker with a PyCharm IDE](https://aws.amazon.com/blogs/machine-learning/run-your-tensorflow-job-on-amazon-sagemaker-with-a-pycharm-ide/) 를 참조 하시면 됩니다. 여기서는 제가 수행한 과정을 보여 드리겠습니다.

### AWS IAM User 및 access key  and secret access key 생성 

AWS Account 를 생성하고, user 를 생성시에 access key ID and secret access key 생성. 이렇게 생성된 access key ID 및 secret access key 는 다른 곳에 저장 해두세요. 이후 설정시에 사용 합니다.
아래는 key 생성 화면을 보여 주고 있습니다.

![create_access_key.png](../imgs/create_access_key.png)

아래는 user 생성을 한 후에, access key 를 보여 주고 있습니다.

![access_key.png](../imgs/access_key.png)


### IAM role 생성, 권한 추가 및 인라인 정책 추가
- 먼저 IAM Role 을 생성 합니다. 가이드: [IAM 역할 생성](https://docs.aws.amazon.com/ko_kr/IAM/latest/UserGuide/id_roles_create.html)
- 이후에 아래 권한 2개를 추가 합니다.
    - SageMaker AmazonSageMakerFullAccess 및 AmazonS3FullAccess 권한에 대한 실행 역할도 필요합니다. SageMaker는 이 역할을 사용하여 SageMaker가 관리하는 AWS 하드웨어에서 사용자 대신 작업을 수행합니다.
- 그리고 이후에 아래 처럼 access_bedrock 인라인 정책의 추가 필요 합니다.    
##### [중요] 아래 ARN 을 복사 해놓으시기 바랍니다. (나중에 필요 합니다.)


![role_bedrock](../imgs/role_bedrock.png)

### 인라인 정책 추가
- 아래와 같이 인라인 정책 생성 (Add permissions - Create inline policy)
- policy name: access_bedrock
- 아래의 JSON 파일을 복사하시고, 콘솔을 열어서 붙여 넣기 하시면 됩니다.


![nn](../imgs/inline-policy.jpg)

```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": [
                "bedrock:*"
            ],
            "Resource": "*",
            "Effect": "Allow",
            "Sid": "BedrockFullAccess"
        }
    ]
}
```

## 1.3. 로컬 컴퓨터에 Access Key 및 Security Key 세팅    

- 로컬 컴퓨터에서 터미널을 오픈합니다. 
- [AWS CLI](https://docs.aws.amazon.com/ko_kr/cli/latest/userguide/getting-started-install.html) 를 설치하고 
- 윈도우 버전 설치 링크 https://awscli.amazonaws.com/AWSCLIV2.msi 
- aws configure를 사용하여 빠르게 구성합니다. (AWS IAM User 를 생성했을시에 카피한 AWS Access Key ID, AWS Secret Access Key 를 입력 합니다.)

```
    d074660793 ~ % aws configure
AWS Access Key ID [****************BIGP]: 
AWS Secret Access Key [****************/5l8]: 
Default region name [us-east-1]: 
Default output format [json]: 
```


## 1.4. Virtual Environment 생성


아래와 같이 터미널을 오픈하고, 가상 환경을 생성 합니다.
window 에서는 anaconda prompt를 검색해서 사용하셔도 됩니다.
![create_virtual_env.png](../imgs/create_virtual_env.png)

## 1.5. Amazon Bedrock Workshop Git 리포 클로닝
특정 원하시는 폴더 위치에 아래 명령어를 실행 하시면 됩니다.
```
git clone https://github.com/dongjin-ml/amazon-bedrock-workshop-webinar-kr.git
```

## 1.6.Model access

사용하고자 하는 Bedrock 의 모델 접근을 설정 합니다.

### Amazon Bedrock Console
![nn](../imgs/model-access/1.png)

### "Get Started"
![nn](../imgs/model-access/2.png)

### "Model access" - "Edit"
사용 모델 활성화 창
![nn](../imgs/model-access/3.png)

### "Save Changes"
사용할 모델을 활성화 후 저장
![nn](../imgs/model-access/4.png)

# 2. Setup 노트북 열기

아래와 같이 합니다.
- VS Code 를 실행합니다.
- 왼쪽 상단의 "폴더 아이콘" 을 클릭하여, 위에서 Git repo 다운로드한 폴더를 선택합니다.
- 폴더에서 00_Setup/setup_local_vscode_bedrock.ipynb 를 클릭하여 오픈 합니다.
- 가상환경이 보이지 않으시면, vscode를 재시작하시면 됩니다.

![open_setup_nb.png](../imgs/open_setup_nb.png)


- 노트북이 열리면, 아래 그림 처럼
    - (1) Select Kernel 을 선택
    - (2) Python Environment 선택
    - (3) 기존에 생성한 Virtual Environment 이름인 bedrock 선택을 합니다.

    
![set_kernel](../imgs/set_kernel.png)

# 3. Setup 노트북 실행

### [중요] 아래 strSageMakerRoleName 에 IAM Role 의 ARN 복사한 것을 붙여 넣기 합니다.

In [1]:
strSageMakerRoleName = "arn:aws:iam::982945618612:user/hwany"  # "arn:aws:iam::{account_number}:role/bedrock_vscode_gsmoon"
print (f"SageMaker Execution Role Name: {strSageMakerRoleName}")

SageMaker Execution Role Name: arn:aws:iam::982945618612:user/hwany


## 3.1. Install python SDK for bedrock

In [2]:
install_needed = True

In [3]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U awscli
    !{sys.executable} -m pip install -U botocore
    !{sys.executable} -m pip install -U boto3
    !{sys.executable} -m pip install -U sagemaker 
    !{sys.executable} -m pip install -U langchain #==0.0.312
    !{sys.executable} -m pip install -U termcolor
    !{sys.executable} -m pip install -U transformers
    !{sys.executable} -m pip install -U librosa
    !{sys.executable} -m pip install -U opensearch-py
    !{sys.executable} -m pip install -U sqlalchemy==2.0.1
    !{sys.executable} -m pip install -U pypdf
    !{sys.executable} -m pip install -U spacy
    !{sys.executable} -m spacy download ko_core_news_md
    !{sys.executable} -m pip install -U ipython
    !{sys.executable} -m pip install -U langchain

    # IPython.Application.instance().kernel.do_shutdown(True)

installing deps and restarting kernel
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
    --------------------------------------- 0.0/2.1 MB 991.0 kB/s eta 0:00:03
   ------ --------------------------------- 0.4/2.1 MB 5.8 MB/s eta 0:00:01
   ----------------------- ---------------- 1.2/2.1 MB 13.0 MB/s eta 0:00:01
   ---------------------------------------  2.1/2.1 MB 16.7 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 15.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.3
    Uninstalling pip-23.3:
      Successfully uninstalled pip-23.3
     ---------------------------------------- 0.0/548.2 kB ? eta -:--:--
     ------ -------------------------------- 92.2/548.2 kB 2.6 MB/s eta 0:00:01
     -------------------------------------- 548.2/548.2 kB 8.5 MB/s eta 0:00:00
     ---------------------------------------- 0.0/83.9 kB ? eta -:--:--
     ---------------------------------------- 83.9/83.9 kB ? eta 0

ERROR: Invalid requirement: '#==0.0.312'


     ---------------------------------------- 0.0/121.5 kB ? eta -:--:--
     --------- ----------------------------- 30.7/121.5 kB 1.3 MB/s eta 0:00:01
     ----------------------------------- -- 112.6/121.5 kB 2.2 MB/s eta 0:00:01
     -------------------------------------- 121.5/121.5 kB 2.4 MB/s eta 0:00:00
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 42.0/42.0 kB ? eta 0:00:00
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 0.0/61.5 kB ? eta -:--:--
     ---------------------------------------- 61.5/61.5 kB 3.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/7.7 MB ? eta -:--:--
   --- ------------------------------------ 0.8/7.7 MB 24.1 MB/s eta 0:00:01
   ----- ---------------------------------- 1.0/7.7 MB 22.0 MB/s eta 0:00:01
   --------- --

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.29.72 requires colorama<0.4.5,>=0.2.5, but you have colorama 0.4.6 which is incompatible.


     ---------------------------------------- 0.0/69.0 MB ? eta -:--:--
     --------------------------------------- 0.0/69.0 MB 640.0 kB/s eta 0:01:48
     --------------------------------------- 0.1/69.0 MB 975.2 kB/s eta 0:01:11
      --------------------------------------- 1.2/69.0 MB 11.0 MB/s eta 0:00:07
     -- ------------------------------------- 4.0/69.0 MB 28.1 MB/s eta 0:00:03
     ----- ---------------------------------- 9.2/69.0 MB 49.1 MB/s eta 0:00:02
     --------- ---------------------------- 16.4/69.0 MB 217.7 MB/s eta 0:00:01
     ------------ ------------------------- 23.1/69.0 MB 217.7 MB/s eta 0:00:01
     ---------------- --------------------- 29.4/69.0 MB 165.0 MB/s eta 0:00:01
     ------------------- ------------------ 36.0/69.0 MB 165.0 MB/s eta 0:00:01
     ---------------------- --------------- 41.2/69.0 MB 162.4 MB/s eta 0:00:01
     ------------------------- ------------ 47.1/69.0 MB 162.4 MB/s eta 0:00:01
     ----------------------------- -------- 53.1

## 3.2. 노트북 환경 세팅

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))

In [8]:
import boto3
import awscli
import botocore
import langchain
from pprint import pprint
from termcolor import colored
from utils.bedrock import bedrock_info

In [9]:
print(f"langchain version check: {langchain.__version__}")
print(f"boto3 version check: {boto3.__version__}")
print(f"botocore version check: {botocore.__version__}")
print(f"awscli version check: {awscli.__version__}")

langchain version check: 0.0.324
boto3 version check: 1.28.72
botocore version check: 1.31.72
awscli version check: 1.29.72


### Region 선택
Region 을 여기서는 "us-east-1" 합니다. 추후에 가용한 Region 을 입력 하시면 됩니다.

In [10]:
bedrock_region = "us-east-1"

### Bedrock Boto3 Client 생성

In [11]:
bedrock_client = boto3.client(
    service_name='bedrock-runtime',
    region_name=bedrock_region,
    endpoint_url=f"https://bedrock-runtime.{bedrock_region}.amazonaws.com"
)

### Bedrock 에서 가용한 모델 보기

In [12]:
print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models())


== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Command': 'cohere.command-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'TBD'}


### 생성 테스트

In [13]:
from langchain.llms.bedrock import Bedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

### Claude v2

In [14]:
llm = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2"),
    client=bedrock_client,
    model_kwargs={
        "max_tokens_to_sample":512,
        "stop_sequences":["\n\nHuman:"],
        "temperature":0,
        "top_p":0.9
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [15]:
prompt = "\n\nHuman:생성형 인공지능의 사례를 말해줘.\n\nAssistant:"

In [16]:
response = llm(prompt)

 네, 생성형 인공지능의 몇 가지 예시를 드리겠습니다.

- 창작 활동: 음악, 문학, 미술 등의 분야에서 인공지능이 새로운 작품을 만드는 것입니다. GPT-3와 같은 자연어 생성 모델, MuseNet과 같은 음악 생성 모델 등이 있습니다.

- 게임 AI: 알파고, 스타크래프트 AI와 같이 게임에서 스스로 전략을 세우고 행동하는 인공지능입니다. 강화학습 등의 기술을 사용합니다.

- 로봇 제어: 자율주행차, 드론과 같이 실세계에서 스스로 판단하고 움직이는 로봇 제어 인공지능입니다. 

- 데이터 생성: GAN(적대적 생성 신경망)등을 이용해 실제와 구분이 어려운 가짜 이미지나 텍스트를 생성하는 인공지능입니다.

- 자연어 처리: 사용자의 질문을 이해하고 적절히 대답하는 인공지능 챗봇도 생성형 인공지능의 한 예시입니다.

이외에도 약물 설계, 재료 설계 등 다양한 분야에서 생성형 인공지능 기술이 활용되고 있습니다.

In [17]:
prompt = "\n\nHuman: 대한 민국의 영토인 독도에 대해서 알려 주세요. \n\nAssistant:"
response = llm(prompt)

 독도에 대한 주요 정보는 다음과 같습니다.

- 독도는 대한민국의 영토로, 경상북도 울릉군에 속하는 섬입니다.

- 독도는 한국어로는 독도, 일본어로는 다케시마라고도 불립니다.

- 독도는 울릉도에서 약 90km 정도 떨어진 동해상에 위치하고 있습니다. 

- 독도는 대한민국이 고유의 영토로 주장하고 있지만, 일본도 영유권을 주장하고 있는 분쟁지역입니다.

- 독도는 지리적으로 울릉도의 부속 도서로 여겨지며, 역사적, 지리적, 국제법적 근거를 바탕으로 대한민국의 고유영토임이 입증되고 있습니다.

- 독도를 둘러싼 한일 간 영유권 분쟁은 여전히 진행 중이며, 대한민국 정부는 독도의 영유권을 강력히 주장하고 있습니다.